Load the forecasts

In [4]:
import pandas as pd

df_forecasts = pd.read_csv("/Users/fohlen/Downloads/export_2024-11-08/merged.csv")
df_forecasts["recorded_at"] = pd.to_datetime(df_forecasts["recorded_at"], utc=True)
df_forecasts["recorded_at"] = df_forecasts["recorded_at"].dt.floor('H')
df_forecasts["forecast_time"] = pd.to_datetime(df_forecasts["forecast_time"], utc=True)
df_forecasts.head()

/var/folders/k6/hdzmrkf915d0twbr_qmhy9l00000gn/T/ipykernel_27133/3023989251.py:5: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_forecasts["recorded_at"] = df_forecasts["recorded_at"].dt.floor('H')


,recorded_at,station_id,forecast_time,parameter,value
0,2024-05-12 15:00:00+00:00,10513,2024-05-12 15:00:00+00:00,precipitation_height_significant_weather_last_1h,0.0
1,2024-05-12 15:00:00+00:00,10513,2024-05-12 16:00:00+00:00,precipitation_height_significant_weather_last_1h,0.0
2,2024-05-12 15:00:00+00:00,10513,2024-05-12 17:00:00+00:00,precipitation_height_significant_weather_last_1h,0.0
3,2024-05-12 15:00:00+00:00,10513,2024-05-12 18:00:00+00:00,precipitation_height_significant_weather_last_1h,0.0
4,2024-05-12 15:00:00+00:00,10513,2024-05-12 19:00:00+00:00,precipitation_height_significant_weather_last_1h,0.0


Load the actual weather

In [2]:
df_measure = pd.read_csv("data/historical_weather.csv")
df_measure["recorded_at"] = pd.to_datetime(df_measure["recorded_at"], utc=True)
df_measure.head()

,recorded_at,station_id,parameter,value
0,2024-05-06 00:00:00+00:00,433,precipitation_height,0.0
1,2024-05-06 01:00:00+00:00,433,precipitation_height,0.0
2,2024-05-06 02:00:00+00:00,433,precipitation_height,0.0
3,2024-05-06 03:00:00+00:00,433,precipitation_height,0.0
4,2024-05-06 04:00:00+00:00,433,precipitation_height,0.0


Mapping from forecast to measurement

In [3]:
reverse_station_mapping = {
    433: 10384, 2667: 10513, 3379: 10865, 1443: 10803, 
    5906: 10729, 2522: 10727, 4104: 10776, 1270: 10554, 
    5419: 10555
}

reverse_parameter_mapping = {
    'precipitation_height': 'precipitation_height_significant_weather_last_1h',
    'temperature_air_mean_2m': 'temperature_air_mean_200'
}

In [48]:
from tqdm import tqdm

regression_rows = []

for index, row in tqdm(df_measure.iterrows(), total=len(df_measure)):
    measurement_time = row["recorded_at"]
    parameter = "temperature_air_mean_2m"

    forecast_rows_measure_time = df_forecasts[
        (df_forecasts["forecast_time"] == measurement_time) &
        (df_forecasts["parameter"] == reverse_parameter_mapping[parameter])
    ]

    for n in range(1, 241):
        filter_time = measurement_time - pd.Timedelta(hours=n)

        forecast_rows = forecast_rows_measure_time[
         (forecast_rows_measure_time["recorded_at"] == filter_time)
        ]

        for index2, forecast_row in forecast_rows.iterrows():
            regression_rows.append((row["value"], measurement_time, parameter))

regression_df = pd.DataFrame(regression_rows, columns=["value", "measure_time", "parameter"])
print(regression_df.head())

  0%|▏                                                                                                                                      | 62/61838 [00:47<13:08:44,  1.31it/s]


KeyboardInterrupt: 

In [37]:
print(rows.station_id.unique())

[ 433 1270 1443 2667 3379 4104 5906]


In [44]:
from sklearn.linear_model import LinearRegression


X_rain = regression_df[regression_df["parameter"] == "temperature_air_mean_2m"].value
y = rows.value.mean()